In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
mnist = input_data.read_data_sets("MNIST_data",one_hot=False)

/home/kenn/anaconda3/envs/tf_base/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
training_epochs = 50
batch_size = 256
display_step = 2
examples_to_show = 10

n_input = 784

In [3]:
X = tf.placeholder(tf.float32,[None,n_input])

on_train = tf.placeholder(tf.bool, name = "on_train")

In [4]:
n_hidden_1 = 256
n_hidden_2 = 64
n_hidden_3 = 10
n_hidden_4 = 2

In [5]:
def fc_layer_with_bn(x_input, num_units, activation, is_training):
    layer = tf.layers.dense(x_input, num_units, use_bias=False)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = activation(layer)
    return layer

def fc_layer_with_bn_without_act(x_input, num_units, is_training):
    layer = tf.layers.dense(x_input, num_units, use_bias=False)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    return layer

In [6]:
# encoder
enc_1 = tf.layers.dense(X,n_hidden_1,tf.nn.relu)
enc_2 = tf.layers.dense(enc_1,n_hidden_2,tf.nn.relu)
enc_3 = tf.layers.dense(enc_2,n_hidden_3,tf.nn.relu)
enc_4 = tf.layers.dense(enc_3,n_hidden_4)
# decoder
dec_1 = tf.layers.dense(enc_4,n_hidden_3,tf.nn.relu)
dec_2 = tf.layers.dense(dec_1,n_hidden_2,tf.nn.relu)
dec_3 = tf.layers.dense(dec_2,n_hidden_1,tf.nn.relu)
y_pred = tf.layers.dense(dec_3,n_input,tf.nn.sigmoid)

y_true = X

In [7]:
cost = tf.reduce_mean(tf.square(y_true - y_pred))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [8]:
init = tf.global_variables_initializer()

In [9]:
%matplotlib
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(mnist.train.num_examples/batch_size)
    for epoch in range(training_epochs):
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _,c = sess.run([optimizer, cost], feed_dict = {X:batch_xs, on_train:True})
            if i < 200 and i%20==0 and epoch==0:
                encoder_result = sess.run(enc_4, feed_dict={X:mnist.test.images, on_train:False})
                plt.clf()
                plt.cla()
                plt.scatter(encoder_result[:,0],encoder_result[:,1],c=mnist.test.labels)
                plt.show()
                plt.pause(0.001)
        if epoch % display_step == 0:
            print("Epoch:","%04d" % (epoch+1),
                 "cost = ","{:.9f}".format(c))
            
            encoder_result = sess.run(enc_4, feed_dict={X:mnist.test.images, on_train:False})
            plt.clf()
            plt.cla()
            plt.scatter(encoder_result[:,0],encoder_result[:,1],c=mnist.test.labels)
            plt.show()
            plt.pause(0.001)
        
        
    print("Optimization Finished!")
    
    

Using matplotlib backend: Qt5Agg
Epoch: 0001 cost =  0.056420039
Epoch: 0003 cost =  0.045878861
Epoch: 0005 cost =  0.044169519
Epoch: 0007 cost =  0.040446494
Epoch: 0009 cost =  0.038275275
Epoch: 0011 cost =  0.039214361
Epoch: 0013 cost =  0.036140721
Epoch: 0015 cost =  0.040290181
Epoch: 0017 cost =  0.037511490
Epoch: 0019 cost =  0.036926180
Epoch: 0021 cost =  0.037123937
Epoch: 0023 cost =  0.036293250
Epoch: 0025 cost =  0.037234850
Epoch: 0027 cost =  0.038132872
Epoch: 0029 cost =  0.036033742
Epoch: 0031 cost =  0.035506696
Epoch: 0033 cost =  0.035177942
Epoch: 0035 cost =  0.035630625
Epoch: 0037 cost =  0.034407664
Epoch: 0039 cost =  0.035006255
Epoch: 0041 cost =  0.036218792
Epoch: 0043 cost =  0.034790449
Epoch: 0045 cost =  0.034848280


KeyboardInterrupt: 